# Analysis of a generic pcap file containing a DNS-based DDoS attack

This is a Jupyter Notebook written for the analysis of generic pcap format files in which there is a DNS-based DDoS attack. If you don't know what is Jupyter Notebook then you must access the documentation (at http://jupyter.org/), learn how to use, install on your own machine, and have fun analysing attacks.

**If you want to do 'this', or 'that', or even 'that other thing' on your analysis or your graphs, believe me: it is possible! GOOGLE IT!**

# Libraries that I use to analyse data

In [ ]:
import pandas as pd #more info at http://pandas.pydata.org/
import numpy as np #more info at http://www.numpy.org/
import matplotlib.pyplot as plt #some examples for you at http://matplotlib.org/gallery.html 
from matplotlib import gridspec #more info at http://matplotlib.org/api/gridspec_api.html
import seaborn as sns

plt.style.use('ggplot') #For improving the visualization style (options: grayscale, bmh, dark_background, ggplot, and fivethirtyeight)

#Magic line: to show the plots inline in the Jupyter Notebook
%matplotlib inline 

import warnings
warnings.filterwarnings('ignore') #To avoid showing annoying warns

# Reading the pcap and Writing into a text file (**VERY expensive TASK!!!)
- If you don't have tshark:
    > sudo apt-get install tshark
- More fields on:https://www.wireshark.org/docs/dfref/
### YOU WILL GET "Error Traversing Database for ipnum...": Don't be worried, it will work!
Inline-style: 

![](figs/boring.gif)
# Don't give up!



In [ ]:
# This operation is extremely timing consuming. At least you only need to do it at ONCE!

!tshark -n -r 'data/!!!FILE_HERE!!!.pcap' -E separator=\;  -E header=y -T fields \
-e frame.time \
-e ip.proto \
-e ip.src \
-e ip.dst \
-e udp.srcport \
-e udp.dstport \
-e tcp.srcport \
-e tcp.dstport \
-e frame.len \
-e dns.qry.type \
-e dns.qry.name \
-e dns.resp.name > pcap.txt

# Reading the converted pcap file (pcap.txt) into a pandas dataframe

In [ ]:
#Data conversion slighty changes the time value
def date_converter(text):
    try:
        return  pd.to_datetime(text)
    except AttributeError:
        return text 

def bytes2bits(value):
    try:
        return  int(value)*8
    except AttributeError:
        return value     

df = pd.read_csv('pcap.txt', error_bad_lines=False, sep=';',\
                  converters = {'frame.time':date_converter, 'frame.len':bytes2bits})

![](figs/boring2.gif)

# Investigating the characteristics of the attack

# Who was the target IP?

In [ ]:
ip_dst=df['ip.dst'].value_counts()
top1_target_ip=ip_dst.index[0]
top2_target_ip=ip_dst.index[1]

target_network=top1_target_ip.split('.')[0]+'.'+top1_target_ip.split('.')[1]
#======================
fig = plt.figure(figsize=(4,4))

ax = plt.subplot2grid((1,1), (0,0))
ip_dst.plot(ax=ax,kind='pie', autopct='%1.1f%%', startangle=270, fontsize=10,title="Target IP")
ax.set_ylabel("")

fig.show()

## Which IP protocols occurred more in the trace?

In [ ]:
ip_proto=df['ip.proto'].value_counts()
top1_target_ip_proto=ip_proto.index[0]
#======================
fig = plt.figure(figsize=(4,4))

ax = plt.subplot2grid((1,1), (0,0))
ip_proto.plot(ax=ax,kind='pie', autopct='%1.1f%%', startangle=270, fontsize=10,title="IP Protocols (number)")
ax.set_ylabel("")

fig.show()

## Distribution of UDP ports for the top 1 (or 2) target IP address(es)

In [ ]:
udp_srcip_dstip_top1=df[df['ip.dst']==top1_target_ip]['ip.src'].value_counts()
udp_srcport_top1=df[df['ip.dst']==top1_target_ip]['udp.srcport'].value_counts()
udp_dstport_top1=df[df['ip.dst']==top1_target_ip]['udp.dstport'].value_counts()

# udp_srcip_dstip_top2=df[df['ip.dst']==top2_target_ip]['ip.src'].value_counts()
# udp_srcport_top2=df[df['ip.dst']==top2_target_ip]['udp.srcport'].value_counts()
# udp_dstport_top2=df[df['ip.dst']==top2_target_ip]['udp.dstport'].value_counts()
#======================
fig = plt.figure(figsize=(12,6))
fig.subplots_adjust(wspace=1)

ax = plt.subplot2grid((2,3), (0,0))
udp_srcip_dstip_top1.plot(kind='pie',ax=ax, autopct='%1.1f%%', startangle=270, fontsize=10,title="Source IPs targetting "+top1_target_ip)
ax.set_ylabel("")

ax1 = plt.subplot2grid((2,3), (0,1))
udp_srcport_top1.plot(kind='pie',ax=ax1, autopct='%1.1f%%', startangle=270, fontsize=10,title="Source UDP Ports Distribution")
ax1.set_ylabel("")

ax2 = plt.subplot2grid((2,3), (0,2))
udp_dstport_top1.plot(kind='pie',ax=ax2, autopct='%1.1f%%', startangle=270, fontsize=10,title="Destination UDP Ports Distribution")
ax2.set_ylabel("")

# ax3 = plt.subplot2grid((2,3), (1,0))
# udp_srcip_dstip_top2.plot(kind='pie',ax=ax3, autopct='%1.1f%%', startangle=270, fontsize=10,title="Source IPs targetting "+top2_target_ip)
# ax3.set_ylabel("")

# ax4 = plt.subplot2grid((2,3), (1,1))
# udp_srcport_top2.plot(kind='pie',ax=ax4, autopct='%1.1f%%', startangle=270, fontsize=10,title="Source UDP Ports Distribution")
# ax4.set_ylabel("")

# ax5 = plt.subplot2grid((2,3), (1,2))
# udp_dstport_top2.plot(kind='pie',ax=ax5, autopct='%1.1f%%', startangle=270, fontsize=10,title="Destination UDP Ports Distribution")
# ax5.set_ylabel("")

fig.show()

# What were the DNS queries used in the attack? and the most used?

In [ ]:
occurrence_dnsquery=df['dns.qry.name'].value_counts()
top_dnsquery=occurrence_dnsquery.index['0']
#======================
fig = plt.figure(figsize=(4,4))
ax = plt.subplot2grid((1,1), (0,0))
occurrence_dnsquery.plot(kind='barh',ax=ax, fontsize=10, title="DNS query")
ax.set_ylabel("")

fig.show()

# What was the greatest frame length that the DNS query used on the attack had? (i.e., which generated the greatest amplification factor)

In [ ]:
top_dnsquery

In [ ]:
max(df['frame.len'][df['dns.qry.name']==top_dnsquery])

# Filtering ONLY the main attack records

In [ ]:
# MANUAL SELECTION: Considering the attack from ONE to ONE port!!!
attack_records=df[df['ip.dst']==top1_target_ip]\
                [df['ip.src'].str.contains(target_network)==False]\
                [df['ip.proto']==top1_target_ip_proto]\
                [df['udp.srcport']==udp_srcport_top1.index[0]]\
                [df['udp.dstport']==udp_dstport_top1.index[0]]
#                 \
#                 [df['dns.qry.name'].str.contains('172-31-28-132')==False]

# # MANUAL SELECTION: Considering the attack from ONE to MANY ports!!!
# attack_records=df[df['ip.dst']==top1_target_ip]\
#                 [df['ip.src'].str.contains(target_network)==False]\
#                 [df['ip.proto']==top1_target_ip_proto]\
#                 [df['udp.srcport']==udp_srcport_top1.index[0]]\
#                 [df['dns.qry.name'].str.contains(top_dnsquery)==True]
                
# # MANUAL SELECTION: Considering the attack from MANY to ONE port!!!
# attack_records=df[df['ip.dst']==top1_target_ip]\
#                 [df['ip.src'].str.contains(target_network)==False]\
#                 [df['ip.proto']==top1_target_ip_proto]\
#                 [df['udp.dstport']==udp_dstport_top1.index[0]]\
#                 [df['dns.qry.name'].str.contains(top_dnsquery)==True]

# # MANUAL SELECTION: Considering the attack from MANY to MANY portS!!!
# attack_records=df[df['ip.dst']==top1_target_ip]\
#                 [df['ip.src'].str.contains(target_network)==False]\
#                 [df['ip.proto']==top1_target_ip_proto]\
#                 [df['udp.srcport']==udp_srcport_top1.index[0]]\
#                 [df['udp.dstport']==udp_dstport_top1.index[0]]\
#                 [df['dns.qry.name'].str.contains(top_dnsquery)==True]

remaining_records=df[~df.isin(attack_records)]


# Time series of the attack and the remaining traffic

In [ ]:
overall_bps=df.set_index(['frame.time']).groupby(pd.TimeGrouper(freq='S')).agg(['sum'])['frame.len']
attack_bps=attack_records.set_index(['frame.time']).groupby(pd.TimeGrouper(freq='S')).agg(['sum'])['frame.len']
attack_bps_median=attack_bps.median()
attack_bps_peak=max(attack_bps['sum'])

remaining_bps=remaining_records.set_index(['frame.time']).groupby(pd.TimeGrouper(freq='S')).agg(['sum'])['frame.len']


#======================
fig = plt.figure(figsize=(12,4))

ax = plt.subplot2grid((1,1), (0,0))

attack_bps.plot(ax=ax, lw=1)
ax.fill_between(attack_bps.index, 0, attack_bps['sum'],color='r')

ax.annotate(str(attack_bps_peak/10e6)+' Mb/s [peak]', (str(attack_bps[attack_bps['sum'] == attack_bps_peak].index.values['0']), attack_bps_peak),
             xytext=(0, 0), textcoords='offset points')
ax.annotate(str(attack_bps_median[0]/10e6)+' Mb/s [median]', (str(attack_bps.index.values['0']), attack_bps_median),
             xytext=(90, 0), textcoords='offset points')

remaining_bps.plot(ax=ax)

ax.legend(['Attacks records','Remaining records'])
ax.set_ylabel("Data [bit]")
ax.set_xlabel("Time [second]")

fig.show()

# How many records in the entire trace, attack records, and remaining records? 

In [ ]:
print 'Records:', len(df),'records (100%),',len(attack_records),'attack records (',len(attack_records)*100/len(df),'%)',',', len(df)-len(attack_records),'remaining records (',(len(df)-len(attack_records))*100/len(df),'% )'

# What is the duration of the entire trace and the attack part? 

In [ ]:
print 'Trace duration:',max(df['frame.time'])-min(df['frame.time'])
print 'Attack duration:',max(attack_records['frame.time'])-min(attack_records['frame.time'])

# How many (source) IPs were involved in the attack?

In [ ]:
len(attack_records['ip.src'].unique())

# What are the top 5 IP addresses that sent more packets?

In [ ]:
top5_srcips_pkts=attack_records['ip.src'].value_counts().head(5).sort_values()
#======================
fig = plt.figure()
ax = plt.subplot2grid((1,1), (0,0))
top5_srcips_pkts.plot(kind='barh')
ax.set_ylabel("Source IP")
ax.set_xlabel("Packets")

fig.show()

# What are the top 5 IP addresses that sent more attack data?

In [ ]:
top5_srcips_bits=attack_records.groupby('ip.src').agg(['sum'])['frame.len'].sort('sum',ascending=False).head(5).sort_values('sum')
#======================
fig = plt.figure()
ax = plt.subplot2grid((1,1), (0,0))
top5_srcips_bits.plot(kind='barh',ax=ax,legend=False)
ax.set_ylabel("Source IP")
ax.set_xlabel("Bits")

fig.show()

# In which country the top 1 source IP address is located?

In [ ]:
!whois -h whois.cymru.com " -v 210.245.0.10"